## Extracting data from a CDP questionnary in docx format

In [1]:
#specific to extracting information from word documents
import os
import zipfile

#other tools useful in extracting the information from our document
import re

#to pretty print our xml:
import xml.dom.minidom
import xml.etree.ElementTree as ET
import PyPDF2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
os.listdir('../Carbometrix/Questionnaires/Automotive')

['Mazda Motor Corporation 2020.docx',
 'Groupe PSA 2020.docx',
 'Hyundai Motor Co 2018.docx',
 'Groupe PSA 2019.docx',
 'Hyundai Motor Co 2020.docx',
 'Honda Motor Co. 2020.docx',
 'Nissan Motor Co., Ltd. 2018.docx',
 'Fiat Chrysler Automobiles NV 2018.docx',
 'Daimler AG 2018.docx',
 'Volkswagen AG 2018.docx',
 'Ford Motor Company 2018.docx',
 'Daimler AG 2020.docx',
 'Isuzu Motors Limited 2018.docx',
 'Renault 2020.docx',
 'SUBARU CORPORATION 2018.docx',
 'PACCAR Inc 2020.docx',
 'PACCAR Inc 2019.docx',
 'Mahindra _ Mahindra 2018.docx',
 'Ford Motor Company 2020.docx',
 'Toyota Motor Corporation 2020.docx',
 'BMW AG 2018.docx',
 'Isuzu Motors Limited 2019.docx',
 'Mitsubishi Motors Corporation 2020.docx',
 'General Motors Company 2020.docx',
 'Mitsubishi Motors Corporation 2019.docx',
 'Renault 2018.docx',
 'BYD 2020.docx',
 'BMW AG 2019.docx',
 'Groupe PSA 2018.docx',
 'Navistar International Corporation 2019.docx',
 'Volkswagen AG 2019.docx',
 'Honda Motor Co. 2018.docx',
 'Hyundai

In [192]:
path = '../Carbometrix/Questionnaires/Automotive/'
data = []

metrics = []
efficiency = []
for file in os.listdir(path):    
    doc_path = path + file
    
    XML_content = generating_XML(doc_path)
    questionnaire = questionnaire_vector(XML_content)
    
    title, sections = section_questionnaire(questionnaire)
    all_subsections = defining_all_subsections(sections)
    
    subsections = text_subsections(sections, all_subsections)
    
    #taking functions
    start_date, end_date = taking_period(subsections)
    scope1 = taking_scope1(subsections)
    #scope2_lb, scope2_bd, scope2_lb_last_year, scope2_mb_last_year = taking_scope2(subsections)
    scope2 = taking_scope2(subsections)
    scope3_names, scope3 = taking_scope3(subsections)
    
    ###testing
#    metrics_name, intensity_metrics = taking_intensity_metrics(subsections)
#    metrics.append([title, intensity_metrics])
#    
#    efficiency_name, efficiency_metrics = taking_efficiency_metrics(subsections)
#    efficiency.append([title, efficiency_metrics])
    
    
    #metrics_names, metrics = taking_intensity_metrics(subsections)
    #metrics.append([title,intensity_metrics])
    #metrics.append(intensity_metrics)
    #metrics = correcting_size(metrics)
    
    #new = metrics_names.copy()
    #for s in metrics_names:
    #    new.append(s)    
    #metrics_names = new.copy()
    
    #efficiency_name, efficiency_metrics = taking_efficiency_metrics(subsections)
    #efficiency.append(efficiency_metrics)
    #efficiecy = correcting_size(efficiency)
    
    
    ############
    #data_file = [title, start_date, end_date, scope1, scope2_location_based, scope2_market_based, scope3, metrics]
    data_file = [title, start_date, end_date, scope1, scope2, scope3]
    data.append(data_file)


In [193]:
all_data = np.array(data, dtype=object).T.tolist()
dic = {}
dic['Title'] = all_data[0]
dic['Start Date'] = all_data[1]
dic['End Date'] = all_data[2]
dic['Scope 1'] = all_data[3]  

all_data[4] = np.array(all_data[4], dtype=object).T
dic['Scope 2 (location based)'] = all_data[4][0]
dic['Scope 2 (market based)'] = all_data[4][1]

all_data[5] = np.array(all_data[5], dtype=object).T
for i in range(len(scope3_names)):
    dic[scope3_names[i]] = all_data[5][i]

df_data = pd.DataFrame.from_dict(dic, orient='index')
df_data = df_data.transpose()

In [180]:
#Saving table in a CSV file 
df_data.to_csv('../Carbometrix/automotive_data.csv')

In [2]:
test = np.random.randint(1,59,5)
print(test)
name = np.array(all_data[0], dtype=object)

[24  9 28 18 54]


NameError: name 'all_data' is not defined

In [196]:
df_data.transpose()

,Title,Start Date,End Date,Scope 1,Scope 2 (location based),Scope 2 (market based),Purchased goods and services,Capital goods,Fuel-and-energy-related activities (not included in Scope 1 or 2),Upstream transportation and distribution,...,Upstream leased assets,Downstream transportation and distribution,Processing of sold products,Use of sold products,End of life treatment of sold products,Downstream leased assets,Franchises,Investments,Other (upstream),Other (downstream)
0,Mazda Motor Corporation - Climate Change 2020,April 1 2019,March 31 2020,117120,534950,<Not Applicable>,4570000,95400,110699,30402,...,nan,63248,nan,31068000,387489,nan,nan,nan,nan,nan
1,Groupe PSA - Climate Change 2020,January 1 2019,December 31 2019,888847,596149,336272,18113776,nan,nan,587046,...,nan,221627,nan,104685836,1043495,nan,nan,nan,nan,nan
2,Hyundai Motor Co - Climate Change 2018,January 1 2017,December 31 2017,805105,1797736,1771132,782672.59,212.65,53863.99,nan,...,nan,748242.1,151545.01,4937006.49,5956.42,1134,nan,363661.6,nan,nan
3,Groupe PSA - Climate Change 2019,janvier 1 2018,d\xc3\xa9cembre 31 2018,878013,435943,307458,22069902,nan,nan,935949,...,nan,335093,nan,88626258,1617689,nan,nan,nan,nan,nan
4,Hyundai Motor Co - Climate Change 2020,January 1 2019,December 31 2019,808139,1900954,<Not Applicable>,1116672,251,97199,nan,...,nan,954579,nan,4399090,8455,4126,nan,394946,nan,nan
5,"Honda Motor Co., Ltd. - Climate Change 2020",April 1 2019,March 31 2020,1240000,3440000,3790000,41200000,1090000,497000,3010000,...,nan,128000,499,247000000,4120000,nan,nan,100000,nan,nan
6,"Nissan Motor Co., Ltd. - Climate Change 2018",April 1 2017,March 31 2018,912476,2563866,2394109,17971000,1203000,412000,740000,...,nan,874000,11000,190261000,425000,470000,nan,nan,nan,nan
7,Fiat Chrysler Automobiles NV - Climate Change ...,January 1 2017,December 31 2017,1101289,3309598,2713748,30536470,1327453,127320,877438,...,0,643850,nan,90152474,889023,0,125535,nan,nan,nan
8,Daimler AG - Climate Change 2018,January 1 2017,December 31 2017,1192000,2041000,1763000,17200000,1068000,192000,251000,...,nan,600000,100000,60200000,1000000,nan,633000,nan,nan,nan
9,Volkswagen AG - Climate Change 2018,January 1 2017,December 31 2017,4297748,5901889,4907050,64139109,13846493,1399741,4374823,...,nan,nan,13000,249466650,1114154,942053,1550000,nan,nan,nan


In [232]:
all_data = np.array(data, dtype=object).T.tolist()
dic = {}
dic['Title'] = all_data[0]
dic['Start Date'] = all_data[1]
dic['End Date'] = all_data[2]
dic['Scope 1'] = all_data[3]    
dic['Scope 2 (location based)'] = all_data[4]
dic['Scope 2 (market based)'] = all_data[5]

#Scope 3
for i in range(len(scope3_names)):
    dic[scope3_names[i]] = np.array(all_data[6], dtype=object).T.tolist()[i]

#METRICS
all_data[7] = correcting_size(all_data[7])
for i in range(len(metrics_names)):
    if i < 10:
        dic['[LDV] '+metrics_names[i]] = np.array(all_data[7], dtype=object).T.tolist()[i]
    else:
        dic['[HDV] '+metrics_names[i]] = np.array(all_data[7], dtype=object).T.tolist()[i]
    

df_data = pd.DataFrame(dic)
#df_data
df_data.to_csv('../Carbometrix/automotive_data.csv')


In [5]:
def generating_XML(doc_path):
    """
    FUNCTION 
    Input: doc_path
    Output: XML_content
    
    Example
    doc_path = '../Carbometrix/General Motor Company 2018.docx'
    """
    
    #Extract the xml file from a docx
    document = zipfile.ZipFile(doc_path)
    XML_content = document.read('word/document.xml')
    
    #The XML content is not the best way. The next lines are necessary to make the XML file more readable
    #uglyXml = xml.dom.minidom.parseString(xml_content).toprettyxml(indent='  ')
    #text_re = re.compile('>\n\s+([^<>\s].*?)\n\s+</', re.DOTALL)    
    #prettyXml = text_re.sub('>\g<1></', uglyXml)

    return XML_content

In [6]:
def questionnaire_vector(XML_content):
    """
    FUNCTION
    Input: XML_content
    Output: questionnaire
    
    This function creates a function of strings of all XML content 
    """
    
    #Taking only w:t strings. 
    questionnaire = re.findall('<w:t xml.*?><', str(XML_content))

    #Removing XML sintax from strings
    for i in range(len(questionnaire)):
        questionnaire[i] = re.sub('<w:t xml:space="preserve">', '', questionnaire[i])
        questionnaire[i] = re.sub('</w:t><','', questionnaire[i])
    
    return questionnaire

In [7]:
def section_questionnaire(questionnaire):
    """
    Organizing our questionary in sections 

    The list 'sections' has the information about each section within the questionary.

    """

    sections_name = ['C0. Introduction', 
                    'C1. Governance', 
                    'C2. Risks and opportunities', 
                    'C3. Business Strategy', 
                    'C4. Targets and performance',
                    'C5. Emissions methodology',
                    'C6. Emissions data',
                    'C7. Emissions breakdowns',
                    'C8. Energy',
                    'C9. Additional metrics',
                    'C10. Verification',
                    'C11. Carbon pricing',
                    'C12. Engagement',
                    'C14. Signoff',
                    ]

    str_quest, sections = [], []
    
    title = questionnaire[0]
    questionnaire = questionnaire[1:]
    sections_name = sections_name[1:] #I need to take of 'C0: Introduction' to guarentee that the first section will be introduction

    for phrase in questionnaire:
        if phrase in sections_name:
            sections.append(str_quest)
            sections_name = sections_name[1:]
            str_quest = []
        str_quest.append(phrase)
        
    return title, sections

In [8]:
def defining_all_subsections(sections):
    subsection_name = []
    subsection = "C"

    for i in range(len(sections)):
        subsection += str(i) + "."
        for j in range(20):
            subsection_name.append(subsection + str(j+1))
        subsection = "C"
    
    all_subsections = []
    for i in range(len(sections)):
        for j in range(len(sections[i])):
            if sections[i][j] in subsection_name:
                all_subsections.append(sections[i][j])
                
    return all_subsections

In [9]:
"""
In order to increase our data specification, in this cell has the list subsections, where is possible to access
each subsection of a section (Introduction, Governance, etc.).

"""
def get_text(text, list_name):
    text = text[1:]
    sub_text = []    
    i = 0
    while text[i] not in list_name:
        sub_text.append(text[i])
        i += 1
        if i >= len(text):
            break
    
    return sub_text

def text_subsections(sections, all_subsections):
    sub_text = []
    subsections = []
    for i in range(len(sections)):
        for j in range(len(sections[i])):
            if sections[i][j] in all_subsections:
                text = sections[i][j:]
                sub_text.append(get_text(text, all_subsections))
                subsections_name = all_subsections[1:]
        subsections.append(sub_text)
        sub_text = []
        
    return subsections



### Taking all data we need 

- Title
- Start and end date
- Scope 1 emissions
- Scope 2 emissions (location-based and market-based
- Scope 3 data (15 + 2 categories)

In [10]:
def get_period(subsections):
    for i in range(len(subsections[0][1])):
        if subsections[0][1][i] == 'Row 1' or subsections[0][1][i] == 'Reporting year':
            start_date = subsections[0][1][i+1]
            end_date = subsections[0][1][i+2]
            
    return start_date, end_date
    

In [11]:
def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

In [184]:
def get_scope1(subsections):
    scope1 = [s for s in subsections[6][0][1:] if isfloat(s)]
    if scope1:
        scope1 = scope1[0]
    else:
        scope1 = np.nan #no answer
    return scope1

In [2]:
def get_scope2(subsections):
    """
    input: a vector with all subsections
    output: 
        scope2[0] = location based 
        scope2[1] = market based
    """
    
    
    scope2 = []

    for i in range(len(subsections[6][2][1:])):
        if subsections[6][2][i] in ['Scope 2, location-based', 'Scope 2, market-based (if applicable)']:
            #print(subsections[6][2][i+1])
            if isfloat(subsections[6][2][i+1]):
                scope2.append(subsections[6][2][i+1])
            elif subsections[6][2][i+1] in ['&lt;Not Applicable&gt;']:
                scope2.append(subsections[6][2][i+1])
            else:
                scope2.append(np.nan)
            
            if len(scope2) == 2:
                break


    scope2 = [s.replace('&lt;', '<') for s in scope2]
    scope2 = [s.replace('&gt;', '>') for s in scope2]
    
    #scope2_lb = scope2[0]  
    #scope2_mb = scope2[1]
    #scope2_lb_last_year = scope2[2]
    #scope2_mb_last_year = scope2[3]
    #return scope2_lb, scope2_mb, scope2_lb_last_year, scope2_mb_last_year
    return scope2

In [191]:
#### C6.5
def get_scope3(subsections):
    """
    ***Sources of Scope 3 emissions***

    0 - Purchased goods and services 
    1 - Capital goods
    2 - Fuel-and-energy-related activities (not included in Scope 1 or 2)
    3 - Upstream transportation and distribution
    4 - Waste generated in operations
    5 - Business travel
    6 - Employee commuting
    7 - Upstream leased assets
    8 - Downstream transportation and distribution
    9 - Processing of sold products
    10 - Use of sold products
    11 - End of life treatment of sold products
    12 - Downstream leased assets
    13 - Franchises
    14 - Investments
    15 - Other (upstream)
    16 - Other (downstream)
    """

    scope3_sections = ['Evaluation status', 'Metric tonnes CO2e', 'Percentage of emissions calculated using data obtained from suppliers or value chain partners']

    scope3_data = []
    for i in range(len(subsections[6][4])):
        if subsections[6][4][i] in scope3_sections:
            if subsections[6][4][i] == 'Evaluation status':
                scope3_data.append(subsections[6][4][i-1])
            elif (subsections[6][4][i] == 'Metric tonnes CO2e' or subsections[6][4][i] == 'Percentage of emissions calculated using data obtained from suppliers or value chain partners'):
                if not(isfloat(subsections[6][4][i+1])):
                    scope3_data.append(np.nan)
                    continue
            scope3_data.append(subsections[6][4][i+1])

    for s in scope3_data:
        if not isfloat(s):
            s.replace('&lt;', '<')
            s.replace('&gt;', '>')

    scope3_data = np.reshape(scope3_data, (17,4)).tolist()

    scope3_names, scope3 = [], []
    for s in scope3_data:
        scope3_names.append(s[0])
        scope3.append(s[2])
    
    return scope3_names, scope3

In [13]:
#scope3_emissions = [[scope3_data[i][0], scope3_data[i][2]] for i in range(len(scope3_data)) if scope3_data[i][2].isdigit()]


#categories, emissions = [], []
#for s in scope3_emissions:
#    categories.append(s[0])
#    emissions.append(int(s[1]))
    
#emissions_sorted_idx = np.argsort(emissions)
#emissions_sorted_idx = emissions_sorted_idx[::-1]
#categories, emissions = np.array(categories), np.array(emissions)


#plt.figure(figsize=(12,12))
#plt.pie(emissions[emissions_sorted_idx], labels=categories[emissions_sorted_idx], autopct='%1.1f%%', radius=1)
#plt.title(title + '\n Scope 3 Emissions (Top 5)')
#plt.show()

In [13]:
#Subsection C-TO7.8
def taking_intensity_metrics(subsections):
    i = 0
    for s in subsections[7][4]:
        i += 1
        if s == 'C-TO7.8':
            C_TO78 = subsections[7][4][i:]
    
    metrics_name = [C_TO78[i] for i in range(1, len(C_TO78), 2)]
    metrics_name = metrics_name[:-1]
    intensity_metrics = [C_TO78[i+1] for i in range(1, len(C_TO78), 2)]
    intensity_metrics = [intensity_metrics[i] for i in range(len(intensity_metrics)) if ((i+1) % 11) != 0]
    
    if not (isfloat(intensity_metrics[9])):
        metric_denominator = float(intensity_metrics[4])
        vehicle_sale = float(intensity_metrics[6])
        vehicle_lifetime = float(intensity_metrics[7])
        annual_distance = float(intensity_metrics[8])
        intensity_metrics[9] = metric_denominator/(vehicle_sale*vehicle_lifetime*annual_distance)

    return metrics_name, intensity_metrics

In [14]:
#Subsection C-TO8.4 or C-TO8.5
def taking_efficiency_metrics(subsections):
    i = 0
    for s in subsections[8][1]:
        i += 1
        if s == 'C-TO8.4' or s == 'C-TO8.5':
            C_TO8 = subsections[8][1][i:]

    efficiency_name = [C_TO8[i] for i in range(1, len(C_TO8), 2)]
    efficiency_metrics = [C_TO8[i+1] for i in range(1, len(C_TO8), 2)]
    efficiency_metrics = [efficiency_metrics[i] for i in range(len(efficiency_metrics)) if ((i+1) % 8) != 0]
    
    return efficiency_name, efficiency_metrics

In [15]:
def correcting_size(vector):
    max_len = 0
    for s in vector:
        if len(s) > max_len:
            max_len = len(s)

    for s in vector: 
        for i in range(max_len - len(s)):
            s.append(' ') #rien répondu
            
    return vector

In [22]:
path = "../Carbometrix/Questionnaires/"
file = "Renault 2019.docx"
#file = "Toyota Motor Corporation 2018.docx"
#file = "Honda Motor Co. 2019.docx"

doc_path = path + file
XML_content = generating_XML(doc_path)
questionnaire = questionnaire_vector(XML_content)

title, sections = section_questionnaire(questionnaire)
all_subsections = defining_all_subsections(sections)

subsections = text_subsections(sections, all_subsections)


In [23]:
path = '../Carbometrix/Questionnaires/'
metrics = []
efficiency = []
for file in os.listdir(path): 
    doc_path = path + file
    XML_content = generating_XML(doc_path)
    questionnaire = questionnaire_vector(XML_content)

    title, sections = section_questionnaire(questionnaire)
    all_subsections = defining_all_subsections(sections)

    subsections = text_subsections(sections, all_subsections)
    
    #####testing
    metrics_name, intensity_metrics = taking_intensity_metrics(subsections)
    metrics.append(intensity_metrics)
    metrics = correcting_size(metrics)
    
    
    efficiency_name, efficiency_metrics = taking_efficiency_metrics(subsections)
    efficiency.append(efficiency_metrics)
    efficiecy = correcting_size(efficiency)
    

In [24]:
intensity_metrics[9]

0.13

In [25]:
metric_denominator = int(intensity_metrics[4])
vehicle_sale = int(intensity_metrics[6])
vehicle_lifetime = int(intensity_metrics[7])
annual_distance = int(intensity_metrics[8])
metric_denominator, vehicle_sale, vehicle_lifetime, annual_distance
intensity_metrics[9] = metric_denominator/(vehicle_sale*vehicle_lifetime*annual_distance)
#load_factor

In [26]:
intensity_metrics[9]

0.13